In [ ]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('PlayTennis.csv')

df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [ ]:
concepts = df.iloc[:,:-1]
target = df.iloc[:,-1]

In [ ]:
class Node:
  def __init__(self,attr):
    self.attr = attr
    self.children = []
    self.outcome = ""
  def __str__(self):
    return "{0} - {1}\n{2}\n".format(self.attr,self.outcome, [c for c in self.children] if len(self.children) > 0 else "")

  def __repr__(self):
    return self.__str__()


In [ ]:
def outcome_count(d):
  p = 0
  n = 0
  for i,r in d.iterrows():
    if target[i] == 'Yes':
      p = p+1
    else:
      n = n+1
  return p,n

def outcome(d):
  p = 0
  n = 0
  for i,r in d.iterrows():
    if target[i] == 'Yes':
      p = p+1
    else:
      n = n+1
  if p == 0:
    return 'No'
  elif n == 0:
    return 'Yes'

def entropy(d,l,outcome=None):
  filtered = d[d[l] == outcome] if outcome else d
  p,n = outcome_count(filtered)
  pr = p/(p+n)
  nr = n/(p+n)
  try:
    return -(pr*math.log(pr,2))-(nr*math.log(nr,2))
  except:
    return 0

def gain(d,l):
  G = entropy(d,l)
  for li in d[l].unique():
    G = G - (d[d[l] == li].shape[0]/d[l].shape[0])*entropy(d,l,li)
  return G

In [ ]:
def max_gain(d):
  gains = { i:gain(d,i) for i,s in d.iteritems() }
  key = ""

  for k,v in gains.items():
    if v == max(gains.values()):
      key = k

  return key

In [ ]:
def ID3(d,target=None):
  tree = Node(target or "")
  out = outcome(d)
  if out:
    tree.outcome = out
    return tree
  elif d.shape[0] <= 1 or d.shape[1] <= 1:
    tree.outcome = max(outcome_count(d))
    return tree
  else:
    tree = Node(max_gain(d))
    for v in d[tree.attr].unique():
      sub_df = d[d[tree.attr] == v].drop(tree.attr,axis=1)
      tree.children.append(ID3(sub_df,v))
    return tree

ID3(concepts)

Outlook - 
[Humidity - 
[High - No

, Normal - Yes

]
, Overcast - Yes

, Wind - 
[Weak - Yes

, Strong - No

]
]